In [11]:
import tensorflow as tf
import numpy as np
from random import randint

num_classes = 5
input = np.random.rand(1, 4, 4, num_classes)
output = np.zeros(shape=(1, 4, 4, num_classes))
sparse_output = np.zeros(shape=(1, 4, 4))

for i in range(0, 4):
    for j in range(0, 4):
        inti = randint(0, num_classes-1)
        sparse_output[0,i,j] = inti
        output[0, i, j][inti] = 1


print(input)
print(output)
print(sparse_output)

x = tf.placeholder(tf.float32, shape=(1, 4, 4, num_classes), name="myInput")
y = tf.placeholder(tf.float32, shape=(1, 4, 4, num_classes), name="myOutput")
sparse_y = tf.placeholder(tf.int32, shape=(1, 4, 4), name="my_sparseOutput")
c_weights = tf.constant([0.4,0.2,0.1,0.15,0.15])


def cross_entropy(softmaxed_output, correct_output, weights, classes):
    with tf.variable_scope("cross_entropy"):
        clip_low = 1e-10
        clip_high = 1
        pixelwise_out = tf.reshape(softmaxed_output, [-1, classes])
        # print(f"shape of pixelwise_out = {pixelwise_out.shape}")
        pixelwise_cor = tf.reshape(correct_output, [-1, classes])
        # print(f"shape of pixelwise_cor = {pixelwise_cor.shape}")

        # return tf.reduce_mean(-tf.reduce_sum(
        #     pixelwise_cor * tf.log(
        #         tf.clip_by_value(pixelwise_out, clip_value_min=clip_low, clip_value_max=clip_high)),
        #     axis=1), name="cross_entropy")

        unweighted = pixelwise_cor * tf.log(tf.clip_by_value(pixelwise_out, clip_value_min=clip_low, clip_value_max=clip_high))
        # print(f"shape of unweighted is {unweighted.shape}"
        weighted = tf.divide(unweighted, weights)
        # print(f"shape of weighted is {weighted.shape}")
        each_pixel_loss = -tf.reduce_sum(weighted, axis=1)
        # print(f"shape of each pixels loss = {each_pixel_loss.shape}")
        return tf.reduce_mean(each_pixel_loss, name="cross_entropy")


def sparse_cross_entropy(softmaxed_output, correct_output, weights, classes):
    with tf.variable_scope("cross_entropy"):
        clip_low = 1e-10
        clip_high = 1

        pixelwise_out = tf.reshape(softmaxed_output, [-1, classes])
        print(f"shape of pixelwise_out = {pixelwise_out.shape}")
        length = int(pixelwise_out.shape[0])
        pixelwise_cor = tf.reshape(correct_output, [-1])

        indices = tf.stack([tf.constant(np.arange(length)), pixelwise_cor], axis=1)
        logit = tf.gather_nd(pixelwise_out, indices)
        dim_weights = tf.gather(weights, pixelwise_cor)

        unweighted = tf.log(tf.clip_by_value(logit, clip_value_min=clip_low, clip_value_max=clip_high))
        weighted = tf.divide(unweighted, dim_weights)
        return -tf.reduce_mean(weighted)

def pixel_wise_softmax(output_map):
    with tf.name_scope("pixel_wise_softmax"):
        max_axis = tf.reduce_max(output_map, axis=3, keepdims=True)
        exponential_map = tf.exp(output_map - max_axis)
        normalize = tf.reduce_sum(exponential_map, axis=3, keepdims=True)
        return exponential_map / normalize


def unweighted_cost(net_output, true_output, classes):
    with tf.variable_scope("unweighted_cost"):
        pixelwise_output = tf.reshape(net_output, [-1, classes])
        pixelwise_correct = tf.reshape(true_output, [-1, classes])
        # print(f"shape of pixelwise_output = {pixelwise_output.shape}")
        # print(f"shape of pixelwise_correct = {pixelwise_correct.shape}")
        cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(
            logits=pixelwise_output, labels=pixelwise_correct))
        # print(f"shape of cost1 = {cost.shape}")
        return cost


def sparse_unweighted_cost(net_output, true_output, classes):
    with tf.variable_scope("unweighted_cost"):
        pixelwise_output = tf.reshape(net_output, [-1, classes])
        pixelwise_correct = tf.reshape(true_output, [-1])
        # print(f"shape of pixelwise_output = {pixelwise_output.shape}")
        # print(f"shape of pixelwise_correct = {pixelwise_correct.shape}")
        cost = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(
            logits=pixelwise_output, labels=pixelwise_correct))
        # print(f"shape of cost1 = {cost.shape}")
        return cost


def sparse_weighted_cost1(net_output, true_output, weights, classes):
    with tf.variable_scope("unweighted_cost"):
        pixelwise_output = tf.reshape(net_output, [-1, classes])
        pixelwise_correct = tf.reshape(true_output, [-1])

        weighted = tf.gather(weights, pixelwise_correct)
        # print(f"shape of pixelwise_output = {pixelwise_output.shape}")
        # print(f"shape of pixelwise_correct = {pixelwise_correct.shape}")
        cost = tf.reduce_mean(tf.divide(
            tf.nn.sparse_softmax_cross_entropy_with_logits(
                logits=pixelwise_output, labels=pixelwise_correct),
            weighted))
        # print(f"shape of cost1 = {cost.shape}")
        return cost


def weighted_cost(net_output, true_output, weights, classes):
    with tf.variable_scope("weighted_cost"):
        softmaxed = tf.nn.softmax(net_output, axis=3)
        cost = cross_entropy(softmaxed, true_output, weights, classes)
        # print(f"shape of softmaxed = {softmaxed.shape}")
        # print(f"shape of cost2 = {cost.shape}")
        return cost


def sparse_weighted_cost2(net_output, true_output, weights, classes):
    with tf.variable_scope("weighted_cost"):
        softmaxed = tf.nn.softmax(net_output, axis=3)
        cost = sparse_cross_entropy(softmaxed, true_output, weights, classes)
        # print(f"shape of softmaxed = {softmaxed.shape}")
        # print(f"shape of cost2 = {cost.shape}")
        return cost



[[[[0.69151476 0.36566605 0.67911265 0.36459452 0.03224102]
   [0.58264637 0.6083964  0.33354919 0.47083659 0.88656647]
   [0.71385716 0.93339825 0.18347469 0.36288919 0.49170572]
   [0.45713387 0.91040412 0.55430057 0.0761062  0.36907429]]

  [[0.33530317 0.33933438 0.1593697  0.46531905 0.43938645]
   [0.34824519 0.30774724 0.8648023  0.67490156 0.55911608]
   [0.12173037 0.98992755 0.80572281 0.68175906 0.68450451]
   [0.34820764 0.47756836 0.25889939 0.62141025 0.94308479]]

  [[0.71380487 0.28275068 0.76839463 0.7055192  0.43732459]
   [0.00165818 0.25647229 0.11308479 0.66844203 0.69137175]
   [0.91821178 0.65036089 0.35145554 0.71842155 0.23343241]
   [0.9380429  0.85470723 0.50471578 0.67144287 0.44734545]]

  [[0.44450301 0.16162256 0.47974544 0.43465014 0.7928301 ]
   [0.08062722 0.45028344 0.40730648 0.85934775 0.77973234]
   [0.53342635 0.55030729 0.43817064 0.22796363 0.04903722]
   [0.52231076 0.35043575 0.17674784 0.27967972 0.80633483]]]]
[[[[0. 1. 0. 0. 0.]
   [0. 0. 1

In [12]:
with tf.Session() as sess:
    onehot_u_cost, onehot_w_cost = sess.run([unweighted_cost(x,y,num_classes), 
                                             weighted_cost(x, y, c_weights, num_classes)], 
                                            feed_dict={x: input, y: output})
    print(f"onehot unweighted cost = {onehot_u_cost}")
    print(f"onehot weighted cost = {onehot_w_cost}")
    sparse_u_cost, sparse_w_cost = sess.run([sparse_unweighted_cost(x, sparse_y, num_classes), 
                                             sparse_weighted_cost1(x, sparse_y, c_weights, num_classes)],
                                            feed_dict={x: input, sparse_y: sparse_output})
    print(f"sparse unweighted cost = {sparse_u_cost}")
    print(f"sparse weighted cost = {sparse_w_cost}")
    sparse_w_cost2 =sess.run([sparse_weighted_cost2(x, sparse_y, c_weights, num_classes)],
                            feed_dict={x: input, sparse_y: sparse_output})
    print(f"my own sparse weighted cost = {sparse_w_cost2}")

onehot unweighted cost = 1.6733899116516113
onehot weighted cost = 12.96287727355957
sparse unweighted cost = 1.6733899116516113
sparse weighted cost = 12.96287727355957
shape of pixelwise_out = (16, 5)
my own sparse weighted cost = [12.962877]
